In [1]:
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

Looking in links: /kaggle/input/lightautoml-038-dependecies
Processing /kaggle/input/lightautoml-038-dependecies/lightautoml-0.3.8-py3-none-any.whl
Processing /kaggle/input/lightautoml-038-dependecies/AutoWoE-1.3.2-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/cmaes-0.10.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/joblib-1.2.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/json2html-1.3.0.tar.gz (from lightautoml==0.3.8)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/lightautoml-038-dependecies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/poetry_core-1.8.1-py3-none-any.whl (from lig

In [2]:
%matplotlib inline
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import copy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import random
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from itertools import combinations  
from scipy.stats import skew
import copy
import joblib
from sklearn.decomposition import PCA
import warnings  
import pickle
from collections import defaultdict
from sklearn.pipeline import Pipeline
import torch
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
warnings.filterwarnings('ignore', category=UserWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
INPUT_DIR = '../input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
train_essays = pd.read_csv('../input/writing-quality-challenge-constructed-essays/train_essays_02.csv')
train_essays.index = train_essays["Unnamed: 0"]
train_essays.index.name = None
train_essays.drop(columns=["Unnamed: 0"], inplace=True)
train_essays_with_upper = pd.read_csv('/kaggle/input/essays-generator-with-upper/essays_with_upper.csv') 
train_essays_with_upper.index = train_essays_with_upper["Unnamed: 0"]
train_essays_with_upper.index.name = None
train_essays_with_upper.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
#essay生成：普通版本和带大写版本
class EssayConstructor:
    
    def processingInputs(self,currTextInput):
        # Where the essay content will be stored
        essayText = ""
        # Produces the essay
        for Input in currTextInput.values:
            # Input[0] = activity
            # Input[1] = cursor_position
            # Input[2] = text_change
            # Input[3] = id
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]              
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')              
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]              
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue                
                
            # If activity = input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        return essayText
            
            
    def getEssays(self,df):
        # Copy required columns
        textInputDf = copy.deepcopy(df[['id', 'activity', 'cursor_position', 'text_change']])
        # Get rid of text inputs that make no change
        textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']     
        # construct essay, fast 
        tqdm.pandas()
        essay=textInputDf.groupby('id')[['activity','cursor_position', 'text_change']].progress_apply(lambda x: self.processingInputs(x))      
        # to dataframe
        essayFrame=essay.to_frame().reset_index()
        essayFrame.columns=['id','essay']
        # Returns the essay series
        return essayFrame

def getEssays_with_upper(df):
    df['down_event_shift'] = df.groupby('id')['down_event'].shift(1)
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change','down_event','down_event_shift']]
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(tqdm(valCountsArr)):
        capital = False
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change','down_event','down_event_shift']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[3] == 'CapsLock':
                capital = not capital
            if Input[0] == 'Nonproduction':
                continue
            if Input[0] != 'Nonproduction':
                if (Input[0] == 'Replace')&(Input[4] == 'Shift'):
                    replaceTxt = Input[2].split(' => ')
                    essayText = essayText[:Input[1] - len(replaceTxt[1])] + (replaceTxt[1]).upper() +\
                    essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                    continue
                    
                if Input[0] == 'Replace':
                    replaceTxt = Input[2].split(' => ')
                    essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] +\
                    essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                    continue
                    
                if Input[0] == 'Paste':
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                    continue
                if Input[0] == 'Remove/Cut':
                    essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                    continue
                if "M" in Input[0]:
                    croppedTxt = Input[0][10:]
                    splitTxt = croppedTxt.split(' To ')
                    valueArr = [item.split(', ') for item in splitTxt]
                    moveData = (int(valueArr[0][0][1:]), 
                                int(valueArr[0][1][:-1]), 
                                int(valueArr[1][0][1:]), 
                                int(valueArr[1][1][:-1]))
                    if moveData[0] != moveData[2]:
                        if moveData[0] < moveData[2]:
                            essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] +\
                            essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                        else:
                            essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] +\
                            essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                    continue
                if capital|((Input[4]=='Shift')&(Input[3]=='q')):
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2].upper() + essayText[Input[1] - len(Input[2]):]
                else:
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        essaySeries[index] = essayText
    essaySeries.index =  textInputDf['id'].unique()
    return pd.DataFrame(essaySeries, columns=['essay'])

In [5]:
# Helper functions
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)
def percentile(n):
    def percentile_(x):
        return x.quantile(n/100)
    percentile_.__name__ = 'pct_{:02.0f}'.format(n)
    return percentile_

In [6]:
def get_essay_aggregations(essay_df):
    cols_to_drop = ['essay']
    # Total essay length
    essay_df['essay_len'] = essay_df['essay'].apply(lambda x: len(x))
    essay_df = essay_df.drop(columns=cols_to_drop)
    return essay_df

def split_essays_into_words(essay_df):
    essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    essay_df = essay_df.explode('word')
    # Word length (number of characters in word)
    essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df['word_len'] != 0]
    return essay_df

def compute_word_aggregations(word_df):
    word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(self.aggregations)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    # New features: computing the # of words whose length exceed word_l
    for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df

def split_essays_into_sentences(essay_df):
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df['sent_word_count_diff'] = essay_df.groupby(level=0)['sent_word_count'].transform(lambda x:np.abs(x.diff()))    

    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(sent_df):
    sent_agg_df = sent_df[['id','sent_len','sent_word_count']].groupby(['id']).agg(self.aggregations)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    # New features: computing the # of sentences whose (character) length exceed sent_l
    for sent_l in [50, 60, 75, 100]:
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_df[sent_df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    return sent_agg_df

def split_essays_into_paragraphs(essay_df):
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of sentences in paragraphs
    essay_df['paragraph_sent_count'] = essay_df['paragraph'].apply(lambda x: len(x.split('\\.|\\?|\\!')))
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(paragraph_df):
    paragraph_agg_df = paragraph_df[['id','paragraph_len', 'paragraph_sent_count', 'paragraph_word_count']].groupby(['id']).agg(self.aggregations)
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    return paragraph_agg_df

In [7]:
# nth percentile function for agg
class Preprocessor:
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes_dict = {
            'q': 'q', 
            ' ': 'space', 
            'NoChange': 'NoChange', 
            '.': 'full_stop', 
            ',': 'comma', 
            '\n': 'newline', 
            "'": 'single_quote', 
            '"': 'double_quote', 
            '-': 'dash', 
            '?': 'question_mark', 
            ';': 'semicolon', 
            '=': 'equals', 
            '/': 'slash', 
            '\\': 'double_backslash', 
            ':': 'colon'
        }
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 70, 100]
        self.percentiles = [5, 10, 25, 50, 75, 90, 95]
        self.percentiles_cols = [percentile(n) for n in self.percentiles]
        self.aggregations = ['mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']
        self.idf = defaultdict(float)
        
        self.essay_constructor = EssayConstructor()
    
    def get_essay_aggregations(self, essay_df):
        cols_to_drop = ['essay']
        # Total essay length
        essay_df['essay_len'] = essay_df['essay'].apply(lambda x: len(x))
        essay_df = essay_df.drop(columns=cols_to_drop)
        return essay_df
    
    def split_essays_into_words(self, essay_df):
        essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
        essay_df = essay_df.explode('word')
        # Word length (number of characters in word)
        essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df['word_len'] != 0]
        return essay_df
    
    def compute_word_aggregations(self, word_df):
        word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(self.aggregations)
        word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
        word_agg_df['id'] = word_agg_df.index
        # New features: computing the # of words whose length exceed word_l
        for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
        word_agg_df = word_agg_df.reset_index(drop=True)
        return word_agg_df
    
    def split_essays_into_sentences(self, essay_df):
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        # Number of words in sentences
        essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        return essay_df

    def compute_sentence_aggregations(self, sent_df):
        sent_agg_df = sent_df[['id','sent_len','sent_word_count']].groupby(['id']).agg(self.aggregations)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index
        # New features: computing the # of sentences whose (character) length exceed sent_l
        for sent_l in [50, 60, 75, 100]:
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_df[sent_df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        return sent_agg_df

    def split_essays_into_paragraphs(self, essay_df):
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        # Number of sentences in paragraphs
        essay_df['paragraph_sent_count'] = essay_df['paragraph'].apply(lambda x: len(x.split('\\.|\\?|\\!')))
        # Number of words in paragraphs
        essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
        
        essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
        return essay_df

    def compute_paragraph_aggregations(self, paragraph_df):
        paragraph_agg_df = paragraph_df[['id','paragraph_len', 'paragraph_sent_count', 'paragraph_word_count']].groupby(['id']).agg(self.aggregations)
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        return paragraph_agg_df
        
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            # make dictionary entry for "move from X to Y"
            di["move_to"] = 0
            
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
                else:
                    # we can do this because there are no missing values
                    di["move_to"] += v
            ret.append(di)
        
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'activity_{act}_tfidf_count' for act in ret.columns]
        normal_cols = [f'activity_{act}_normal_count' for act in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'{colname}_{event}_tfidf_count' for event in ret.columns]
        normal_cols = [f'{colname}_{event}_normal_count' for event in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes_dict.keys():
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'text_change_{self.text_changes_dict[txt_change]}_tfidf_count' for txt_change in ret.columns]
        normal_cols = [f'text_change_{self.text_changes_dict[txt_change]}_normal_count' for txt_change in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg
    
    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret

    # Credit: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs/notebook
    def make_space_features(self, df):
        df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
        df['time_diff'] = abs(df['down_time'] - df['up_time_lagged']) / 1000

        group = df.groupby('id')['time_diff']
        largest_lantency = group.max()
        smallest_lantency = group.min()
        median_lantency = group.median()
        initial_pause = df.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        
        result = pd.DataFrame({
            'id': df['id'].unique(),
            'largest_lantency': largest_lantency,
            'smallest_lantency': smallest_lantency,
            'median_lantency': median_lantency,
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True)
        return result
    
    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        for percentile in self.percentiles:
            tmp_df[f'input_word_length_pct_{percentile}'] = tmp_df['text_change'].apply(lambda x: np.percentile([len(i) for i in x] if len(x) > 0 else 0, 
                                                                                                               percentile))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df: pd.DataFrame, save_essays_path: str):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get essay feats
        print("Getting essays")
        essay_df = self.essay_constructor.getEssays(df)
        essay_df.to_csv(save_essays_path, index=False)

        print("Getting essay aggregations data")
        essay_agg_df = self.get_essay_aggregations(essay_df)
        feats = feats.merge(essay_agg_df, on='id', how='left')

        print("Getting essay word aggregations data")
        word_df = self.split_essays_into_words(essay_df)
        word_agg_df = self.compute_word_aggregations(word_df)
        feats = feats.merge(word_agg_df, on='id', how='left')

        print("Getting essay sentence aggregations data")
        sent_df = self.split_essays_into_sentences(essay_df)
        sent_agg_df = self.compute_sentence_aggregations(sent_df)
        feats = feats.merge(sent_agg_df, on='id', how='left')

        print("Getting essay paragraph aggregations data")
        paragraph_df = self.split_essays_into_paragraphs(essay_df)
        paragraph_agg_df = self.compute_paragraph_aggregations(paragraph_df)
        feats = feats.merge(paragraph_agg_df, on='id', how='left')
        
        # engineer counts data
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        # space features
        print("Engineering space-related data")
        tmp_df = self.make_space_features(df)
        feats = feats.merge(tmp_df, on='id', how='left')
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)
        
        # cursor position shift
        print("Engineering cursor position data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)
        
        # word count shift
        print("Engineering word count data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        percentiles_cols = [percentile(n) for n in self.percentiles]
        feats_stat = [
            ('action_time',self.percentiles_cols),
            ('activity', ['nunique']),
            ('down_event', [ 'nunique']),
            ('up_event', [ 'nunique']),
            ('text_change', [ 'nunique']),
            ] 

        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['first','last', 'max', 'min', 'mean', 'std', 'sem', 'skew', pd.DataFrame.kurt]+ percentiles_cols),
                (f'cursor_position_change{gap}', ['first','last','max', 'mean', 'std','sem', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'sum', 'sem', 'skew', pd.DataFrame.kurt] + percentiles_cols),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left') 

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        print("Done!")
        return feats

In [8]:
#bruteforce agg
train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)

test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
    ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

In [9]:
def duration_features(train_logs,test_logs):
    tr_logs,te_logs = copy.deepcopy(train_logs),copy.deepcopy(test_logs)
    data = []
    for logs in [tr_logs, te_logs]:
        #logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
        #logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000
        logs['time_diff'] = logs.groupby('id')['down_time'].diff()
        group = logs.groupby('id')['time_diff']
        largest_lantency = group.max()
        smallest_lantency = group.min()
        median_lantency = group.median()
        initial_pause = logs.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        data.append(pd.DataFrame({
            'id': logs['id'].unique(),
            'largest_lantency': largest_lantency,
            'smallest_lantency': smallest_lantency,
            'median_lantency': median_lantency,
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True))
    return data

In [10]:
def essay_CountVectorizer(train_essays,test_essays):
    train_essaysdf = copy.deepcopy(train_essays['essay'])
    test_essaysdf = copy.deepcopy(test_essays['essay'])
    train_essaysdf = pd.DataFrame({'id': train_essaysdf.index, 'essay': train_essaysdf.values})
    test_essaysdf = pd.DataFrame({'id': test_essaysdf.index, 'essay': test_essaysdf.values})
    merged_data = train_essaysdf.merge(train_scores, on='id')

    count_vectorizer = CountVectorizer(ngram_range=(1, 2))
    X_tokenizer_train = count_vectorizer.fit_transform(merged_data['essay'])
    X_tokenizer_test = count_vectorizer.transform(test_essaysdf['essay'])
    y = merged_data['score']

    X_tokenizer_train = X_tokenizer_train.todense()
    X_tokenizer_test = X_tokenizer_test.todense()

    train_count_vector = pd.DataFrame()
    test_count_vector = pd.DataFrame()
    for i in range(X_tokenizer_train.shape[1]) : 
        L = list(X_tokenizer_train[:,i])
        li = [int(x) for x in L ]
        train_count_vector[f'feature {i}'] = li
    for i in range(X_tokenizer_test.shape[1]): 
        L = list(X_tokenizer_test[:,i])
        li = [int(x) for x in L ]
        test_count_vector[f'feature {i}'] = li

    df_train_index = train_essaysdf['id']
    df_test_index = test_essaysdf['id']
    train_count_vector.loc[:, 'id'] = df_train_index
    test_count_vector.loc[:, 'id'] = df_test_index

    #保留部分特征，保留90%以上的人出现过的特征
    save_cols = []
    for i in train_count_vector.columns:
        if sum(train_count_vector[i]==0)/len(train_count_vector)<0.1:
            save_cols.append(i)
    return train_count_vector,test_count_vector,save_cols

In [11]:
%%time
def other_features(df,method='train'):
    a = pd.DataFrame()
    a['Input_all_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['activity']!='Input'))/df.groupby(['id']).apply(lambda x:sum(x['activity']=='Input'))
    a['all_q_ratio'] = df.groupby(['id']).apply(lambda x:sum(x['down_event']!='q'))/df.groupby(['id']).apply(lambda x:sum(x['down_event']=='q'))
    activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
    events_dict = {
                    'q':'q', 
                    'Space':'Space', 
                    'Backspace':'Backspace', 
                    'Shift':'Shift', 
                    'ArrowRight':'ArrowRight', 
                    'Leftclick':'Leftclick', 
                    'ArrowLeft':'ArrowLeft', 
                    '.':'fullstop', 
                    ',':'comma', 
                    'ArrowDown':'ArrowDown', 
                    'ArrowUp':'ArrowUp', 
                    'Enter':'Enter', 
                    'CapsLock':'CapsLock', 
                    "'":'single_quote', 
                    'Delete':'Delete', 
                    'Unidentified':'Unidentified',
                  }
    
    for i in tqdm(activities):
        for j in events_dict:
            a[f'{i}_{events_dict[j]}_count'] = df.groupby('id').apply(lambda x:len(x[(x['activity']==i)&(x['down_event']==j)]))
    return a.reset_index()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [12]:
def language_error(df,df_essays):
    a = pd.DataFrame()
    df['down_event_shift'] = df.groupby('id')['down_event'].shift(-1)
    #letter_upper = df.groupby('id').apply(lambda x:len(x[(x['down_event']=='CapsLock')|((x['down_event']=='Shift')&(x['down_event_shift']=='q'))]))
    letter_upper = df.groupby('id').apply(lambda x:len(x[(x['down_event']=='CapsLock')])//2+len(x[(x['down_event']=='Shift')&(x['down_event_shift']=='q')]))
    a['letter_big_count'] = letter_upper.values
    a['id'] = df['id'].unique()
    
    essay_df = copy.deepcopy(df_essays)
    essay_df['id'] = essay_df.index
    
    #避免将qqq.).切分成多个句子
    #essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'\.\]|\.\)|\.\}|\?\]|\?\)|\?\}|\!\]|\!\)|\!\}','qq',x))
    
    essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())    
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df['sent_len']!=0]
    errors_num = (essay_df.groupby('id').apply(len)-letter_upper).values
    a['error_num'] = errors_num                          #如果句子个数大于大写字母按键次数，那么文章会有语法错误
    return a
def sentence_error(df):
    essay_df = copy.deepcopy(df)
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    essay_df = essay_df[essay_df['paragraph_len']!=0]
    essay_df['only_space'] = essay_df['paragraph'].apply(lambda x:'q' not in x)
    essay_df = essay_df[essay_df['only_space']==False]
    a = pd.DataFrame()
    a['para_error'] = essay_df.groupby('id').apply(lambda x:len(x[x['paragraph_len']<25]))   #一个段落字符过少可能不是完整的一句话，可能存在语法错误

    return a.reset_index()

def language_error_letter(df):
    essay_df = copy.deepcopy(df)
    essay_df['id'] = essay_df.index
    essay_df['essay'] = essay_df['essay'].apply(lambda x:re.sub(r'q\.q\.','qqq',x))
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')   #explode将列表里的元素展开
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip()) 
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    essay_df['language_error_letter'] = essay_df['sent'].apply(lambda x:x[0])
    essay_df['if_q'] = essay_df['language_error_letter'].apply(lambda x:x.lower()=='q')
    essay_df = essay_df[essay_df['if_q']==True]
    a = pd.DataFrame()
    a['language_error_letter'] = essay_df.groupby('id').apply(lambda x:len(x[x['language_error_letter']=='q']))
    return a.reset_index()


punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/','@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
def calculate_pauses(df, pause_threshold=2000):
    # Compute IKI within each 'id' group
    df['IKI'] = df.groupby('id')['down_time'].diff()

    # Define pauses
    df['is_pause'] = (df['IKI'] > pause_threshold)

    # Compute statistics for IKI
    iki_stats = df.groupby('id')['IKI'].agg(['mean', 'median', 'std', 'max']).reset_index().rename(columns={
        'mean': 'iki_mean',
        'median': 'iki_median',
        'std': 'iki_std',
        'max': 'iki_max'
    })

    # Compute pause counts
    pause_counts = df.groupby('id')['is_pause'].sum().reset_index(name='pause_count')

    # Compute average pause time excluding NaNs
    pause_times = df[df['is_pause']].groupby('id')['IKI'].mean().reset_index(name='average_pause_time')

    # Compute total pause time for paragraph
    para_pause_duration = df.groupby('id').apply(lambda group: group['IKI'].where(group['text_change'] == '\n').sum()).reset_index(name='para_pause_duration')

    # Merge pause features
    pause_features = pause_counts.merge(pause_times, on='id', how='left')
    pause_features = pause_features.merge(para_pause_duration, on='id', how='left')
    pause_features = pause_features.merge(iki_stats, on='id', how='left')

    # Compute total IKI time and exclude NaNs
    total_time = df.groupby('id')['IKI'].sum().reset_index(name='total_time')

    # Merge the total time into pause_features
    pause_features = pause_features.merge(total_time, on='id', how='left')

    # Calculate pause time ratio
    pause_features['pause_time_ratio'] = pause_features['pause_count'] * pause_features['average_pause_time']
    pause_features['pause_time_ratio'] = pause_features['pause_time_ratio'] / pause_features['total_time'].replace(0, np.nan)

    # Calculate times between sentences within each 'id' group
    df['sentence_end_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].isin(['.', '?', '!']))).reset_index(level=0, drop=True)

    # Calculate statistics for times between sentences
    between_sentences_stats = df.groupby('id')['sentence_end_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_sentences_IKI', 'std': 'std_between_sentences_IKI'})

    # Calculate within-word IKI for 'q' characters within each 'id'
    df['within_word_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'] == 'q')).reset_index(level=0, drop=True)

    # Calculate statistics for within-word IKI
    within_word_stats = df.groupby('id')['within_word_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_within_word_IKI', 'std': 'std_within_word_IKI'})

    # Calculate between-words IKI for spaces or punctuation followed by 'q'
    df['between_words_IKI'] = df.groupby('id').apply(lambda group: group['down_time'].diff().where(group['text_change'].shift().isin([' '] + punctuations) & (group['text_change'] == 'q'))).reset_index(level=0, drop=True)

    # Calculate statistics for between-words IKI
    between_words_stats = df.groupby('id')['between_words_IKI'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'mean_between_words_IKI', 'std': 'std_between_words_IKI'})

    # Combine all the IKI related features into one DataFrame
    pause_features = pause_features.merge(between_sentences_stats, on='id', how='left')
    pause_features = pause_features.merge(within_word_stats, on='id', how='left')
    pause_features = pause_features.merge(between_words_stats, on='id', how='left')

    return pause_features


def R_burst(df):
    a = pd.DataFrame()
    df = df[(df['activity']=='Input')|(df['activity']=='Remove/Cut')].reset_index(drop=True)
    df['activity_shift'] = df.groupby('id')['activity'].shift().fillna(method='bfill')
    df['is_R_burst'] = df['activity'] != df['activity_shift']
    a['revision_count'] = df.groupby('id').apply(lambda x:x['is_R_burst'].sum())
    df['keystroke_duration'] = df.groupby('id')['down_time'].diff()
    df = df[df['keystroke_duration'].notnull()]

    a['revision_count_above2s'] = df.groupby('id').apply(lambda x:x[(x['is_R_burst']==True)&(x['keystroke_duration']>2)]['is_R_burst'].sum()).values
    Rburst =  df[(df['is_R_burst']==True)&(df['keystroke_duration']>2)]   #&(df['keystroke_duration']>2)
    Rburst_statistic = Rburst.groupby('id').agg({'keystroke_duration':['mean','max','sum','median']})
    Rburst_statistic.columns = ['_'.join(x) for x in Rburst_statistic.columns]

    return a.merge(Rburst_statistic.reset_index(),on='id',how='left')



In [13]:
use_pre_fe_data = False
if use_pre_fe_data:
    pass
else:
    preprocessor_train = Preprocessor(seed = 42)
    train_feats = preprocessor_train.make_feats(train_logs,save_essays_path = 'train_essays.csv')
    del preprocessor_train
    gc.collect()

preprocessor_test = Preprocessor(seed = 42)
test_feats = preprocessor_test.make_feats(test_logs,save_essays_path = 'test_essays.csv')
print('<train_feats and test_feats done.>')

train_feats = train_feats.merge(train_agg_fe_df, on='id', how='left')
test_feats = test_feats.merge(test_agg_fe_df, on='id', how='left')
print('<merge train and test agg done.>')

train_eD592674, test_eD592674 = duration_features(train_logs,test_logs)
train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
train_feats = train_feats.merge(train_scores, on='id', how='left')
print('<merge duration features|train label done.>')

test_essays = (EssayConstructor().getEssays(test_logs))
test_essays.index = test_essays['id'].values
test_essays.drop(columns='id',inplace=True)
train_count_vector,test_count_vector,save_cols = essay_CountVectorizer(train_essays,test_essays)
train_feats = train_feats.merge(train_count_vector[save_cols], on='id', how='left')
test_feats = test_feats.merge(test_count_vector[save_cols], on='id', how='left')
print('<merge train and test essay CountVectorizer done.>')

if os.path.exists('/kaggle/input/lgbm-and-nn-on-sentences'):  
    train_agg_ratio = pd.read_csv('/kaggle/input/lgbm-and-nn-on-sentences/train_agg_ratio.csv')
else:
    train_agg_ratio = other_features(train_logs)
#删除掉全部为0的列
save_cols = []
for i in train_agg_ratio.columns:
    if sum(train_agg_ratio[i]==0)<len(train_agg_ratio):
        save_cols.append(i)
test_agg_ratio = other_features(test_logs)                          
train_feats = train_feats.merge(train_agg_ratio[save_cols], on='id', how='left')
test_feats = test_feats.merge(test_agg_ratio[save_cols], on='id', how='left')
train_agg_ratio.to_csv('/kaggle/working/train_agg_ratio.csv',index=0)
print('<merge other featuers done.>')

tr_language_error_agg = language_error(train_logs,train_essays)
te_language_error_agg = language_error(test_logs,test_essays)
train_feats = train_feats.merge(tr_language_error_agg, on='id', how='left')
test_feats = test_feats.merge(te_language_error_agg, on='id', how='left')

tr_sentence_error_agg = sentence_error(train_essays)
te_sentence_error_agg = sentence_error(test_essays)
train_feats = train_feats.merge(tr_sentence_error_agg, on='id', how='left')
test_feats = test_feats.merge(te_sentence_error_agg, on='id', how='left')

tr_language_error_letter_agg =  language_error_letter(train_essays_with_upper)
test_essays_with_upper = getEssays_with_upper(test_logs)
te_language_error_letter_agg =  language_error_letter(test_essays_with_upper)
train_feats = train_feats.merge(tr_language_error_letter_agg, on='id', how='left')
test_feats = test_feats.merge(te_language_error_letter_agg, on='id', how='left')
print('<merge language errors done.>')

tr_IKI_agg = calculate_pauses(train_logs)
te_IKI_agg = calculate_pauses(test_logs)
train_feats = train_feats.merge(tr_IKI_agg, on='id', how='left')
test_feats = test_feats.merge(te_IKI_agg, on='id', how='left')
print('<merge IKI features done.>')

tr_rburst_agg = R_burst(train_logs)
te_rburst_agg = R_burst(test_logs)
train_feats = train_feats.merge(tr_rburst_agg, on='id', how='left')
test_feats = test_feats.merge(te_rburst_agg, on='id', how='left')
print('<merge R_burst features done.>')

Starting to engineer features
Getting essays


100%|██████████| 2471/2471 [00:16<00:00, 151.88it/s]


Getting essay aggregations data
Getting essay word aggregations data
Getting essay sentence aggregations data
Getting essay paragraph aggregations data
Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 6033.01it/s]


Engineering event counts data


...

<merge train and test essay CountVectorizer done.>


100%|██████████| 5/5 [00:00<00:00,  8.54it/s]


<merge other featuers done.>


100%|██████████| 3/3 [00:00<00:00, 675.09it/s]


<merge language errors done.>
<merge IKI features done.>
<merge R_burst features done.>


In [14]:
target_col = ['score']
drop_cols = ['id']
has_inf_cols = ['activity_Remove/Cut_tfidf_count',
                 'activity_Replace_tfidf_count',
                 'activity_Paste_tfidf_count',
                 'activity_move_to_tfidf_count',
                 'down_event_Backspace_tfidf_count',
                 'down_event_Shift_tfidf_count',
                 'down_event_ArrowRight_tfidf_count',
                 'down_event_Leftclick_tfidf_count',
                 'down_event_ArrowLeft_tfidf_count',
                 'down_event_._tfidf_count',
                 'down_event_,_tfidf_count',
                 'down_event_ArrowDown_tfidf_count',
                 'down_event_ArrowUp_tfidf_count',
                 'down_event_Enter_tfidf_count',
                 'down_event_CapsLock_tfidf_count',
                 "down_event_'_tfidf_count",
                 'down_event_Delete_tfidf_count',
                 'down_event_Unidentified_tfidf_count',
                 'up_event_Backspace_tfidf_count',
                 'up_event_Shift_tfidf_count',
                 'up_event_ArrowRight_tfidf_count',
                 'up_event_Leftclick_tfidf_count',
                 'up_event_ArrowLeft_tfidf_count',
                 'up_event_._tfidf_count',
                 'up_event_,_tfidf_count',
                 'up_event_ArrowDown_tfidf_count',
                 'up_event_ArrowUp_tfidf_count',
                 'up_event_Enter_tfidf_count',
                 'up_event_CapsLock_tfidf_count',
                 "up_event_'_tfidf_count",
                 'up_event_Delete_tfidf_count',
                 'up_event_Unidentified_tfidf_count',
                 'text_change_comma_tfidf_count',
                 'text_change_newline_tfidf_count',
                 'text_change_single_quote_tfidf_count',
                 'text_change_double_quote_tfidf_count',
                 'text_change_dash_tfidf_count',
                 'text_change_question_mark_tfidf_count',
                 'text_change_semicolon_tfidf_count',
                 'text_change_equals_tfidf_count',
                 'text_change_slash_tfidf_count',
                 'text_change_double_backslash_tfidf_count',
                 'text_change_colon_tfidf_count']

drop_cols_NN = []
for i in train_feats.columns:
    if 'action_time' in i:
        drop_cols_NN.append(i)
    if 'up_time' in i:
        drop_cols_NN.append(i)
cols = [col for col in train_feats.columns if col not in drop_cols+drop_cols_NN+has_inf_cols]  

In [15]:
TASK = 'reg'
task = Task(TASK)
TIMEOUT = 10000
N_THREADS = 2
TRAIN_BS = 128
USE_PLR = True
USE_QNT = True
N_FOLDS = 5
RANDOM_STATE = 42
ADVANCED_ROLES = False
TARGET_NAME = 'score'
roles = {'target':TARGET_NAME}

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [16]:
def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return log_loss(y_true, y_pred)
    elif task.name == 'reg' or task.name == 'multi:reg':
        return mean_squared_error(y_true,y_pred,squared=False)
    else:
        raise 'Task is not correct.'
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'

In [17]:
OOF_PREDS = np.zeros(len(train_feats))
algo = "mlp"
for RANDOM_STATE in range(42,43):
    automl = TabularAutoML(
        task = task, 
        timeout = TIMEOUT,
        cpu_limit = N_THREADS,
        general_params = {"use_algos": [[algo]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
        nn_params = {
            "n_epochs": 8, 
            "bs": TRAIN_BS, 
            "num_workers": 0, 
            "path_to_save": None, 
            "freeze_defaults": True, 
            "cont_embedder": use_plr(USE_PLR), 
            "hidden_size": 32,
            'clip_grad': True, 
            'clip_grad_params': {'max_norm': 1},
            'snap_params': {'early_stopping':False,'swa':False},
        },
        nn_pipeline_params = {"use_qnt": USE_QNT, "use_te": False},
        reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': ADVANCED_ROLES},
    )
    
    OOF_PREDS += take_pred_from_task((automl.fit_predict(train_data=train_feats[cols],roles = {'target':'score'}, verbose = 3)).data, task)  /len(range(42,43))
    joblib.dump(automl,f'MLP_{RANDOM_STATE}.pkl')

[04:04:44] Stdout logging level is INFO3.
[04:04:44] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[04:04:44] Task: reg

[04:04:44] Start automl preset with listed constraints:
[04:04:44] - time: 10000.00 seconds
[04:04:44] - CPU: 2 cores
[04:04:44] - memory: 16 GB

[04:04:44] Train data shape: (2471, 519)

[04:04:44] Layer 1 train process start. Time left 9999.31 secs
[04:04:46] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 ...
[04:04:46] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 =====
[04:04:51] Epoch: 0, train loss: 0.9906322956085205, val loss: 0.7656087875366211, val metric: -0.7684003710746765
[04:04:56] Epoch: 1, train loss: 0.6874363422393799, val loss: 0.42412498593330383, val metric: -0.42373931407928467
[04:05:01] Epoch: 2, train loss: 0.44750791788101196, val loss: 0.31821590662002563, val metric: -0.3170606791973114
[04:05:06] Epoch: 3, train loss: 0.3931441009044647, val loss: 0.2995024621486

In [18]:
metrics.mean_squared_error(train_feats['score'],OOF_PREDS,squared=False)   #10:0.6030168941831987

0.598726656665098

In [19]:
test_cols = [col for col in train_feats.columns if col not in drop_cols+drop_cols_NN+target_col]

In [20]:
TEST_PREDS = np.zeros(len(test_feats))
for i in range(42,43):
    automl = joblib.load('/kaggle/working/MLP_{}.pkl'.format(i))
    TEST_PREDS += automl.predict(test_feats[test_cols]).data[:, 0] / len(range(42,43))

In [21]:
test_feats['score'] = TEST_PREDS
test_feats[['id', 'score']].to_csv("submission.csv", index=False)
test_feats[['id', 'score']]

/tmp/ipykernel_26/3044189139.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_feats['score'] = TEST_PREDS


,id,score
0,0000aaaa,1.480401
1,2222bbbb,0.606703
2,4444cccc,0.533156


In [22]:
has_inf_cols = []
for i in cols:
    has_inf = np.isinf(train_feats[i].values).any()
    if has_inf:
        print(f'feature{i} has inf')
        has_inf_cols.append(i)

In [23]:
has_inf_cols

[]